In [1]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from utils.split_df import train_test_split

# Датасет: Mercedes-Benz Greener Manufacturing

Этот набор данных содержит анонимизированный набор переменных, каждая из которых представляет пользовательскую функцию в автомобиле Mercedes. Например, переменной может быть 4WD, дополнительная пневмоподвеска или головной дисплей.

Абсолютная величина обозначена буквой "y" и представляет собой время (в секундах), которое потребовалось автомобилю для прохождения тестирования по каждому набору пременных.

Для текущей работы используется только часть датасета в файле train.csv

In [2]:

data_path= "mercedes-benz-greener-manufacturing/train.csv"
df=pd.read_csv(data_path,index_col='ID').reset_index(drop=True)
df.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,k,v,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
1,88.53,k,t,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
2,76.26,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 377 entries, y to X385
dtypes: float64(1), int64(368), object(8)
memory usage: 12.1+ MB


Датасет состоит из целевой переменной 'y' - которая представлена числом типа float64
и колонками с предикторами 368 из которых явлются бинарными признаками  а 8 категориальными

In [4]:
print('Any null values',df.isnull().any().any())

Any null values False


датасет не имеет пропущенных значений

In [5]:
# Делим на тренировочную и тестовою выборки
train_raw,test_raw=train_test_split(df,0.2)

# Тренировка моделей RandomForest и XGBoost

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor

# Определение колонок с категориальными и числовыми данными
cat_columns=[col for col in df.columns if df[col].dtype=='O']
num_columns=[col for col in df.columns if df[col].dtype=='int64']


# Создание пайплайна
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_columns)
    ])

pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', RandomForestRegressor())])
pipeline_gb = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', XGBRegressor())])

# Обучение моделей

pipeline_rf.fit(train_raw.drop('y', axis=1), train_raw['y'])
pipeline_gb.fit(train_raw.drop('y', axis=1), train_raw['y'])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['X10', 'X11', 'X12', 'X13',
                                                   'X14', 'X15', 'X16', 'X17',
                                                   'X18', 'X19', 'X20', 'X21',
                                                   'X22', 'X23', 'X24', 'X26',
                                                   'X27', 'X28', 'X29', 'X30',
                                                   'X31', 'X32', 'X33', 'X34',
                                                   'X35', 'X36', 'X37', 'X38',
                                                   'X39', 'X40', ...]),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['X0', 'X1', 'X2', 'X3', 'X4',
                                                   'X5', '...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [7]:
from joblib import dump
# Сохранение моделей
dump(pipeline_rf, 'model/pipeline_rf.joblib')
dump(pipeline_gb, 'model/pipeline_gb.joblib')

['model/pipeline_gb.joblib']

# Обучение H2O AutoMl

In [8]:
import h2o
h2o.init()
from h2o.automl import H2OAutoML
# создаем экземпляр H2OAutoML

automl = H2OAutoML(max_models=20,max_runtime_secs=3600, seed=42)

train_h2o=h2o.H2OFrame(train_raw)
test_h2o=h2o.H2OFrame(test_raw)

predictors = train_raw.columns.drop('y').to_list()
response = "y"

# запускаем автоматический отбор моделей и обучение
automl.train(x=predictors, y=response, training_frame=train_h2o,
               validation_frame = test_h2o)

# Выбор лучшей модели\
auto_model=automl.get_best_model()
# save the model
model_path = h2o.save_model(model=auto_model, path="model", force=True)
print(model_path)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,12 hours 3 mins
H2O_cluster_timezone:,Asia/Krasnoyarsk
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_roman_yy9t75
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.282 Gb
H2O_cluster_total_cores:,24
H2O_cluster_allowed_cores:,24
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
22:19:47.84: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
22:19:47.93: _train param, Dropping bad and constant columns: [X107, X297, X330, X233, X93, X289, X235, X257, X268, X95, X347, X190, X11, X290, X260, X293]

██
22:19:56.798: _train param, Dropping bad and constant columns: [X107, X297, X330, X233, X93, X289, X235, X257, X268, X95, X347, X190, X11, X290, X260, X293]


22:19:58.283: _train param, Dropping bad and constant columns: [X107, X297, X330, X233, X93, X289, X235, X257, X268, X95, X347, X190, X11, X290, X260, X293]


22:20:00.926: _train param, Dropping bad a

# Обучение модели FLAML AutoML

In [21]:
from flaml import AutoML
import joblib

# Initialize an AutoML instance
flaml_automl = AutoML()
# Specify automl goal and constraint
automl_settings = {
    "time_budget": 100,  # in seconds
    "metric": 'mse',
    "task": 'regression',
    "log_file_name": "automl.log",
}
X_train=train_raw.drop('y', axis=1)
y_train = train_raw['y']
# Train with labeled input data
flaml_automl.fit(X_train=X_train, y_train=y_train,
           **automl_settings)

# Сохраняем модель в файл
joblib.dump(flaml_automl, 'model/flaml_model.joblib')


[flaml.automl.logger: 04-16 22:54:45] {1768} INFO - task = regression
[flaml.automl.logger: 04-16 22:54:45] {1775} INFO - Data split method: uniform
[flaml.automl.logger: 04-16 22:54:45] {1778} INFO - Evaluation method: holdout
[flaml.automl.logger: 04-16 22:54:45] {1891} INFO - Minimizing error metric: mse
[flaml.automl.logger: 04-16 22:54:45] {2011} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 04-16 22:54:45] {2341} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 04-16 22:54:46] {2479} INFO - Estimated sufficient time budget=1728s. Estimated necessary time budget=12s.
[flaml.automl.logger: 04-16 22:54:46] {2526} INFO -  at 1.1s,	estimator lgbm's best error=109.9795,	best estimator lgbm's best error=109.9795
[flaml.automl.logger: 04-16 22:54:46] {2341} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 04-16 22:54:46] {2526} INFO -  at 1.2s,	estimator lgbm's best error=109.9795,	bes

['model/flaml_model.joblib']

# Сравнение моделей

In [22]:
from joblib import load
#загрузка сохраненных моделей

# Загрузить модель случайного леса
pipeline_rf = load('model/pipeline_rf.joblib')

# Загрузить модель градентного бустинга 
pipeline_gb = load('model/pipeline_gb.joblib')

# Загрузить модель H2O AutoMl
auto_model = h2o.load_model('model/StackedEnsemble_BestOfFamily_1_AutoML_9_20230416_214946')

# Загрузить модель FLAML AutoMl 
flaml_model = load('model/flaml_model.joblib')




In [26]:
# Предсказания по тестовой выборке

pred_rf=pipeline_rf.predict(test_raw.drop('y', axis=1))
pred_gb=pipeline_gb.predict(test_raw.drop('y', axis=1))
pred_h2o_auto=auto_model.predict(test_h2o).as_data_frame().values
pred_flaml=flaml_model.predict(test_raw.drop('y', axis=1))

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/home/roman/projects/hw1_H2O/henv/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'X2' has levels not trained on: ["aa", "am", "o"]
  warnings.warn(w)
/home/roman/projects/hw1_H2O/henv/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'X5' has levels not trained on: ["h", "y"]
  warnings.warn(w)


In [27]:
from sklearn.metrics import r2_score,mean_squared_error

mse_rf = mean_squared_error(test_raw['y'].values, pred_rf)
mse_gb = mean_squared_error(test_raw['y'].values, pred_gb)
mse_h2o = mean_squared_error(test_raw['y'].values, pred_h2o_auto)
mse_flaml = mean_squared_error(test_raw['y'].values, pred_flaml)

r2score_rf = r2_score(test_raw['y'].values, pred_rf)
r2score_gb = r2_score(test_raw['y'].values, pred_gb)
r2score_h2o = r2_score(test_raw['y'].values, pred_h2o_auto)
r2score_flaml = r2_score(test_raw['y'].values, pred_flaml)

print(f'Mean Squared Error Random Forest: {mse_rf:.2f}')
print(f'Mean Squared Error Gradient Busting: {mse_gb:.2f}')
print(f'Mean Squared Error H2O AutoML: {mse_h2o:.2f}')
print(f'Mean Squared Error FLAML AutoML: {mse_flaml:.2f}')
print('------------------------------------------------')
print(f'R2 Score Random Forest: {r2score_rf:.2f}')
print(f'R2 Score Gradient Busting: {r2score_gb:.2f}')
print(f'R2 Score H2O AutoML: {r2score_h2o:.2f}')
print(f'R2 Score FLAML AutoML: {r2score_flaml:.2f}')

Mean Squared Error Random Forest: 76.02
Mean Squared Error Gradient Busting: 82.66
Mean Squared Error H2O AutoML: 64.87
Mean Squared Error FLAML AutoML: 64.27
------------------------------------------------
R2 Score Random Forest: 0.51
R2 Score Gradient Busting: 0.47
R2 Score H2O AutoML: 0.58
R2 Score FLAML AutoML: 0.59


# Выводы

Проведено сравнение двух библиотек AutoML (FLAML и H2O) и 2 моделей RF и XGBoost "из коробки", без обработки параметров и данных. 

Обе библиотеки AutoMl показали результаты превосходящие результаты моделей "из коробки". Сранение производилось по метрикам MSE и R2 Score